### Objectives
1. Get 50 frames from each video
2. At a rate of 1 frame per second of video

In [5]:
import os
import moviepy.editor as mpe
from tqdm import tqdm
from pytube import YouTube
from pandas import read_csv
from zipfile import ZipFile
os.chdir('video')

In [6]:
FILE='hardhatvids'
VID_PATH_GET='hardhatvids/'
VID_PATH_SAVE='video_grabs/' # folder to save images in

#### Option: Add videos to folder

In [ ]:
a_vids=read_csv('video_lists/vid_a.csv', header=None, names=['links'])
for v in tqdm(a_vids.links):
    try:
        YouTube(v).streams.filter(subtype='mp4').first().download()
    except:
        print(v, ' is not available.')

#### Option - Unzip downloaded mp4 files by nb

In [ ]:
with ZipFile(FILE, 'r') as zf:
    zf.extractall(VID_PATH_SAVE)

#### Clip videos and grab frames

In [25]:
# get all .mp4 files for processing
lst=os.listdir(VID_PATH_GET) # folder with .mp4 files 
vidlst=sorted([i for i in lst if '.mp4' in i])

In [27]:
# make/clear frame save dir
try:
    os.mkdir(VID_PATH_SAVE)
except FileExistsError as f:
    print(f)
    pass
    
# clear out save directory for frames
inpt=input('Do you want to delete all .jpeg from directory? y/n ')
if inpt=='y': [os.remove(i) for i in os.listdir('video_grabs') if '.jpeg' in i]

[Errno 17] File exists: 'video_grabs/'
Do you want to delete all .jpeg from directory? y/n y


In [28]:
# get a 50 sec clip with one frame per second with some buffer to start (e.g 20 secs), if the vid is long enough
max_segment_length=50; start=10



err_lst,vid_proc=[],[] # list of vids not processed, list of processed vids

counter=0
for vid in tqdm(vidlst):
    video = mpe.VideoFileClip(VID_PATH_GET+vid)
    start=1 if video.duration < max_segment_length else video.duration-max_segment_length
    finish=max_segment_length+start if video.duration > max_segment_length+start else video.duration
    
    for t in range(0,int(video.duration),max(1,int(video.duration/50))):
        try:
            video.save_frame(VID_PATH_SAVE+'{}_{}.jpeg'.
                             format(vid.split('.mp4')[0].strip(), counter), t=t) # save frame at t=2 as JPEG
            vid_proc.append((vid,': ',video.duration))
        except:
            err_lst.append('{} had OS error and was not processed'.format(vid))
            pass
        
        counter+=1

100%|██████████| 20/20 [02:09<00:00,  7.82s/it]


In [19]:
print('Total videos processed: ', len(vidlst)-len(set(err_lst)))
print('Total frames created: ', len(os.listdir('video_grabs')))
print('Videos not processed:\n', set(err_lst))

NameError: name 'err_lst' is not defined

In [4]:
frames = sorted(os.listdir('video_grabs'))
andrew=frames[:400]
sudha=frames[400:800]
silas=frames[800:]
batches={'andrew':andrew,'sudha':sudha,'silas':silas}

FileNotFoundError: [Errno 2] No such file or directory: 'video_grabs'

In [ ]:
for name,batch in batches.items():    
    with ZipFile('{}.zip'.format(name),'w') as zip: 
        # writing each file one by one 
        for file in batch: 
            zip.write(VID_PATH_SAVE+file)

#### helpers

In [ ]:
# use to erase all jpg from file with lst = os.listdir():
#[os.remove(i) for i in os.listdir('video_grabs') if '.jpeg' in i]

In [ ]:
# use to erase all jpg from file:
# set([os.remove(i) for i in lst if '.png' in i])

In [ ]:
# yt.streams.filter(progressive=True, file_extension='mp4').desc().all()

In [ ]:
# option to automate download of vids
"""
import dropbox
dbx = dropbox.Dropbox("accesstokenhere")
dbx.files_download_to_file(j, '/'+ j)
"""